In [ ]:
!pip uninstall paperflow -y

In [ ]:
 !pip install paperflow[extraction-all,rag,providers] --upgrade

In [ ]:
import paperflow
paperflow.__version__

# PaperFlow: Complete Academic Paper Processing Demo

PaperFlow is a unified pipeline for academic paper ingestion, extraction, and RAG (Retrieval-Augmented Generation). It allows you to:

- **Search** across multiple academic sources (arXiv, PubMed, Semantic Scholar, OpenAlex)
- **Download** PDFs automatically
- **Extract** text and structure from PDFs
- **Chunk** content for RAG applications
- **Embed** and store in vector databases
- **Query** papers using natural language

## Features

- 🔍 Multi-source search (arXiv, PubMed, Semantic Scholar, OpenAlex)
- 📥 Automatic PDF downloading
- 📄 Advanced PDF text extraction (Marker AI, Docling, MarkItDown)
- ✂️ Intelligent text chunking
- 🧠 Vector embeddings for RAG
- 💾 ChromaDB integration
- 🔗 LangChain compatibility
- 📊 Tabular result display
- 💻 Command-line interface

## Installation

Install the latest version of PaperFlow with all optional dependencies.

In [ ]:
from paperflow import PaperPipeline

# Configuration
USE_GPU = False  # Set to True if you have CUDA GPU and want faster extraction
PDF_DIR = './test_pdfs'

# Create pipeline
pipeline = PaperPipeline(gpu=USE_GPU, pdf_dir=PDF_DIR)

# Search for papers
print('Searching for papers on transformers...')
results = pipeline.search('transformer attention', sources=['pubmed'], max_results=3)

# Display results
print(results)
print()

# Download all papers first
print('Downloading all papers...')
papers = []
for i, paper_meta in enumerate(results.papers, 1):
    print(f'Downloading paper {i}: {paper_meta["title"][:50]}...')
    paper = pipeline.process(paper_meta, download=True, extract=False, chunk=False, embed=False)
    papers.append(paper)
    print(f'  - PDF saved: {paper.pdf_path}')
    print()

# Then extract all papers
print('Extracting all papers...')
for i, paper in enumerate(papers, 1):
    print(f'Extracting paper {i}: {paper.metadata.title[:50]}...')
    paper = pipeline.process(paper, download=False, extract=True, chunk=True, embed=False)
    print(f'  - Sections: {len(paper.sections)}, Chunks: {len(paper.chunks)}')
    print()

print('All done!')

## What Happens Next?

After processing, you can:

- **Query the papers**: Use RAG to ask questions about the content
- **Export to LangChain**: Get LangChain documents for further processing
- **Save to vector database**: Store embeddings for semantic search
- **Analyze content**: Access extracted sections, chunks, and metadata

## Command Line Usage

You can also use paperflow from the command line:

```bash
# Install with CLI support
pip install paperflow

# Search and display results
paperflow "transformer attention" --sources pubmed --max-results 5
```

## Advanced Usage

For more advanced features, install optional dependencies:

```bash
pip install paperflow[all]  # Full installation
pip install paperflow[extraction]  # PDF extraction only
pip install paperflow[rag]  # RAG features only
```

Check out the [documentation](https://github.com/osllmai/paperflow) for more examples!

In [ ]:
from paperflow import PaperPipeline

# Configuration
USE_GPU = False  # Set to True if you have CUDA GPU and want faster extraction
PDF_DIR = './test_pdfs'

# Choose PDF extraction backend:
# - "auto": Try marker → docling → markitdown (recommended)
# - "marker": High quality, best for academic papers, GPU support
# - "docling": Good table/figure extraction, IBM, GPU support  
# - "markitdown": Lightweight, fast, CPU only, Microsoft
EXTRACTION_BACKEND = "auto"

print(f"GPU acceleration: {'Enabled' if USE_GPU else 'Disabled'}")
print(f"PDF directory: {PDF_DIR}")
print(f"Extraction backend: {EXTRACTION_BACKEND}")

In [ ]:
from paperflow import PaperPipeline

# Create pipeline with selected backend
pipeline = PaperPipeline(
    gpu=USE_GPU, 
    pdf_dir=PDF_DIR,
    extraction_backend=EXTRACTION_BACKEND
)

print(f"✅ Pipeline created with {pipeline._extractor.active_backend} backend")

## 1. arXiv Provider Demo

Search for computer science papers on arXiv, the preprint server for physics, mathematics, computer science, and more.

In [ ]:
# Search arXiv
print('🔍 Searching arXiv for "transformer attention" papers...')
results = pipeline.search('transformer attention mechanism', sources=['arxiv'], max_results=3)

print(f"✅ Found {results.total_found} papers in {results.search_time_ms}ms")

In [ ]:
# Display arXiv results
if results.papers:
    print("\n📋 arXiv Search Results:")
    print("-" * 80)
    
    for i, paper in enumerate(results.papers, 1):
        title = paper['title'][:70] + "..." if len(paper['title']) > 70 else paper['title']
        authors = ", ".join([a['name'] for a in paper.get('authors', [])[:2]])
        if len(paper.get('authors', [])) > 2:
            authors += " et al."
        
        print(f"{i}. {title}")
        print(f"   Authors: {authors}")
        print(f"   Year: {paper.get('year', 'N/A')}")
        print(f"   arXiv ID: {paper.get('arxiv_id', 'N/A')}")
        print(f"   Citations: {paper.get('citation_count', 0)}")
        print()
else:
    print("❌ No papers found")

In [ ]:
# Display arXiv results in JSON format
if results.papers:
    import json
    print("📋 arXiv Search Results in JSON:")
    print(json.dumps(results.papers, indent=2))

## 2. PubMed Provider Demo

Search for biomedical research papers in PubMed, the premier database for biomedical literature.

In [ ]:
# Search PubMed
print('🔍 Searching PubMed for "CRISPR gene editing" papers...')
pubmed_results = pipeline.search('CRISPR gene editing therapy', sources=['pubmed'], max_results=3)

print(f"✅ Found {pubmed_results.total_found} papers in {pubmed_results.search_time_ms}ms")

In [ ]:
# Display PubMed results
if pubmed_results.papers:
    print("\n📋 PubMed Search Results:")
    print("-" * 80)
    
    for i, paper in enumerate(pubmed_results.papers, 1):
        title = paper['title'][:70] + "..." if len(paper['title']) > 70 else paper['title']
        authors = ", ".join([a['name'] for a in paper.get('authors', [])[:2]])
        if len(paper.get('authors', [])) > 2:
            authors += " et al."
        
        print(f"{i}. {title}")
        print(f"   Authors: {authors}")
        print(f"   Year: {paper.get('year', 'N/A')}")
        print(f"   DOI: {paper.get('doi', 'N/A')}")
        print(f"   Citations: {paper.get('citation_count', 0)}")
        print()

In [ ]:
# Display PubMed results in JSON format
if pubmed_results.papers:
    import json
    print("📋 PubMed Search Results in JSON:")
    print(json.dumps(pubmed_results.papers, indent=2))

## 3. Semantic Scholar Provider Demo

Search using Semantic Scholar's AI-powered academic search engine.

In [ ]:
# Search Semantic Scholar
print('🔍 Searching Semantic Scholar for "large language models"...')
sem_results = pipeline.search('large language models GPT', sources=['semantic_scholar'], max_results=3)

print(f"✅ Found {sem_results.total_found} papers in {sem_results.search_time_ms}ms")

In [ ]:
# Display Semantic Scholar results
if sem_results.papers:
    print("\n📋 Semantic Scholar Results:")
    print("-" * 80)
    
    for i, paper in enumerate(sem_results.papers, 1):
        title = paper['title'][:70] + "..." if len(paper['title']) > 70 else paper['title']
        authors = ", ".join([a['name'] for a in paper.get('authors', [])[:2]])
        if len(paper.get('authors', [])) > 2:
            authors += " et al."
        
        print(f"{i}. {title}")
        print(f"   Authors: {authors}")
        print(f"   Year: {paper.get('year', 'N/A')}")
        print(f"   DOI: {paper.get('doi', 'N/A')}")
        print(f"   Citations: {paper.get('citation_count', 0)}")
        print()

In [ ]:
# Display Semantic Scholar results in JSON format
if sem_results.papers:
    import json
    print("📋 Semantic Scholar Search Results in JSON:")
    print(json.dumps(sem_results.papers, indent=2))

## 4. OpenAlex Provider Demo

Search the OpenAlex catalog, which covers millions of scholarly works from all disciplines.

In [ ]:
# Search OpenAlex
print('🔍 Searching OpenAlex for "climate change adaptation"...')
openalex_results = pipeline.search('climate change adaptation strategies', sources=['openalex'], max_results=3)

print(f"✅ Found {openalex_results.total_found} papers in {openalex_results.search_time_ms}ms")

In [ ]:
# Display OpenAlex results
if openalex_results.papers:
    print("\n📋 OpenAlex Results:")
    print("-" * 80)
    
    for i, paper in enumerate(openalex_results.papers, 1):
        title = paper['title'][:70] + "..." if len(paper['title']) > 70 else paper['title']
        authors = ", ".join([a['name'] for a in paper.get('authors', [])[:2]])
        if len(paper.get('authors', [])) > 2:
            authors += " et al."
        
        print(f"{i}. {title}")
        print(f"   Authors: {authors}")
        print(f"   Year: {paper.get('year', 'N/A')}")
        print(f"   DOI: {paper.get('doi', 'N/A')}")
        print(f"   Citations: {paper.get('citation_count', 0)}")
        print()

In [ ]:
# Display OpenAlex results in JSON format
if openalex_results.papers:
    import json
    print("📋 OpenAlex Search Results in JSON:")
    print(json.dumps(openalex_results.papers, indent=2))

## 5. Complete Pipeline Demo

Demonstrate the full pipeline: search → download → extract → chunk → embed.

In [ ]:
# Full pipeline demonstration
print("🚀 Running complete pipeline...")

# 1. Search
print("1. Searching for papers...")
search_results = pipeline.search("neural networks", sources=["arxiv"], max_results=1)

if search_results.papers:
    paper_dict = search_results.papers[0]
    
    # 2. Download
    print("2. Downloading PDF...")
    paper = pipeline.download(paper_dict)
    
    # 3. Extract
    print("3. Extracting content...")
    paper = pipeline.extract(paper)
    
    # 4. Chunk
    print("4. Creating chunks...")
    paper = pipeline.chunk(paper)
    
    # 5. Embed (if embeddings available)
    print("5. Creating embeddings...")
    try:
        paper = pipeline.embed(paper)
        print("✅ Pipeline completed successfully!")
        
        # Show results
        print(f"\n📊 Results:")
        print(f"Title: {paper.metadata.title[:50]}...")
        print(f"Sections: {len(paper.sections)}")
        print(f"Chunks: {len(paper.chunks)}")
        print(f"Has embeddings: {paper.has_embeddings}")
        
    except Exception as e:
        print(f"⚠️ Embedding failed (missing dependencies): {e}")
        print("✅ Pipeline completed (without embeddings)")
        
        print(f"\n📊 Results:")
        print(f"Title: {paper.metadata.title[:50]}...")
        print(f"Sections: {len(paper.sections)}")
        print(f"Chunks: {len(paper.chunks)}")
else:
    print("❌ No papers found for pipeline demo")

## Summary

This notebook demonstrated:

✅ **All 4 academic providers**: arXiv, PubMed, Semantic Scholar, OpenAlex  
✅ **Search across all sources** with formatted and JSON output  
✅ **Complete processing pipeline**: search → download → extract → chunk → embed  
✅ **PDF extraction backends**: Marker AI, Docling, MarkItDown with auto-fallback  
✅ **GPU acceleration** support for faster processing  

### Key Features Used:
- Unified search interface across all providers
- Automatic PDF downloading and text extraction
- Intelligent text chunking for RAG
- Vector embeddings for semantic search
- Tabular and JSON result display
- Error handling and graceful fallbacks

Happy researching! 🔬📚